In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
class DecisionTreeRegressor:
  def fit(self, X, y, min_leaf):
    self.dtree = Node(X, y, np.array(np.arange(len(y))), min_leaf)
    return self

  def predict(self, X):
    return self.dtree.predict(X.values)

In [ ]:
class Node:

  def __init__(self, x, y, idxs, min_leaㄹ):
    self.x = x
    self.y = y
    self.idxs = idxs
    self.min_leaf = min_leaf
    self.row_count = len(idxs)
    self.col_count = x.shape[1]
    self.val = np.mean(y[idxs])
    self.score = float('inf')
    self.find_varsplit()

  # split main function
  def find_varsplit(self):
    for var_idx in range(self.col_count): 
      x = self.x.values[self.idxs, var_idx]

      # 최적의 조합 찾기
      for r in range(self.row_count):
        lhs = x<=x[r]
        rhs = x>x[r]
        if rhs.sum() < self.min_leaf or lhs.sum() < self.min_leaf: continue
        # 최소 min_leaf의 개수만큼은 있어야 한다
        # min_leaf의 개수보다 적으면 밑의 내용은 continue를 통해 skip


        curr_score = self.find_score(lhs, rhs)
        if curr_score < self.score:
          self.var_idx = var_idx
          self.score = curr_score
          self.split = x[r]



    if self.is_leaf: return
    x = self.split_col
    lhs = np.nonzero(x<=self.split)[0]
    rhs = np.nonzero(x>self.split)[0]
    # self.split: node split의 기준이 되는 값
    self.lhs = Node(self.x, self.y, self.idxs[lhs], self.min_leaf)
    self.rhs = Node(self.x, self.y, self.idxs[rhs], self.min_leaf)
    # self.split을 기준으로 좌우 트리 분할하기


   

  # low standard deviation for split 
  # split that minimizes the weighted averages of the std = minimize RMSE
  def find_score(self, lhs, rhs):
    y = self.y[self.idxs]
    lhs_std = y[lhs].std()
    rhs_std = y[rhs].std()
    return lhs_std*lhs.sum() + rhs_std * rhs.sum()


  @property
  def split_col(self): return self.x.values[self.idxs, self.var_idx]

  @property
  def is_leaf(self): return self.score == float('inf')

  def predict(self, x):
    return np.array([self.predict_row(xi) for xi in x])


  def predict_row(self, xi):
    if self.is_leaf: return self.val
    # self.val = np.mean(y[idxs])
    # is_leaf의 경우에는 yi return
    node = self.lhs if xi[self.var_idx] <=self.split else self.rhs
    return node.predict_row(xi)

In [22]:
!ls '/content/drive/My Drive/expedia/expedia-personalized-sort/data'

Desktop.zip	test3_data_test.csv   test.csv
test3_data	test3_data_train.csv  train_10000.csv
test3_data.csv	test3.zip	      train.csv


In [ ]:
df_train = pd.read_csv('/content/drive/My Drive/expedia/expedia-personalized-sort/data/test3_data_train.csv')

In [ ]:
X_train = df_train.drop(['price_usd'], axis='columns')
y_train = df_train['price_usd']
# turn DataFrame into numpy ndarray for computation using .values

In [ ]:
df_test = pd.read_csv('/content/drive/My Drive/expedia/expedia-personalized-sort/data/test3_data_test.csv')

In [ ]:
X_test = df_test.drop(['price_usd'], axis='columns')
y_test = df_test['price_usd']
# turn DataFrame into numpy ndarray for computation using .values

In [ ]:
model = DecisionTreeRegressor()

In [12]:
model.fit(X_train, y_train)

In [ ]:
# r2_score function
def r2_score(y_true, y_pred):
    SS_total = 0
    SS_reg = 0
    
    for i in range(len(y_true)):
        x = (y_true[i] - y_pred[i])**2
        SS_reg = SS_reg+x
    
    y_true_mean = np.mean(y_true)
    for i in range(len(y_true)):
        x = (y_true[i]-y_true_mean)**2
        SS_total = SS_total+x
        
    
    print(1 - (SS_reg/SS_total))

In [ ]:
prediction = model.predict(X_test)

In [16]:
  pd.DataFrame({
    'y_true': y_test,
    'y_pred' : np.ravel(prediction)
})

,y_true,y_pred
0,29.75,19.285714
1,149.00,129.000000
2,129.00,140.600000
3,96.00,78.600000
4,135.00,108.000000
...,...,...
9995,115.00,83.250000
9996,119.00,92.500000
9997,44.50,41.250000
9998,44.50,54.400000


In [17]:
r2_score(y_test, prediction)

0.5695908782000201
